# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis y visualización de datos

### Mentoría 16 - Trabajo práctico 1

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---

## Librerías

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
pd.options.display.float_format = '{:,.2f}'.format

## Dataset

In [ ]:
url = 'https://www.dropbox.com/scl/fi/iaagjtks3apflrywvomuv/muestra_diplodatos_ventas_2023.csv?dl=1&rlkey=zfsh0bnwbomd4g56bcjysytiy'
ventas = pd.read_csv(url)

---
# Paso 1: Carga y exploración inicial de datos

### Variables

Hay un total de 19 variables. Significado tentativo:
*   **ID_VENDEDOR**: identificación del vendedor, anonimizado mediante hasheo. Tipo: objeto.
*   **INSCRIPCION**: desconocemos. Tipo: entero.
*   **AÑO**: año de imputación del registro. Tipo: entero.
*   **MES**: mes de imputación del registro. Tipo: entero.
*   **CATEGORIA**: descripción del Código de Actividad Codiguero NAES Año 2018 y siguientes. Tipo: objeto.
*   **DEPOSITO**: desconocemos. Tipo: entero.
*   **DESCRIPCION_CATEGORIA**: subrubro definido por la DGR (Dato Interno, no declarado por el contribuyente). Tipo: objeto.
*   **TOTAL_VENTAS**: monto total de ventas (base imponible). Tipo: flotante.
*   **PORCENTAJE_COMISION_EMPRESA**: comisión por las ventas en la plataforma (alícuota). Tipo: flotante.
*   **COMISION_EMPRESA**: comisión por las ventas en la plataforma (valor). Tipo: flotante.
*   **TRATAMIENTO_FISCAL**: indica qué tratamiento fiscal se le da a la operación, especialmente respecto a la alícuota a cobrarle. Tipo: objeto.
*   **DESC_TRATAMIENTO_FISCAL**: categoriza los valores de `TRATAMIENTO_FISCAL`. Tipo: objeto.
*   **TRATAMIENTO_DIFERNCIAL**: indica el artículo de alguna reglamentación aplicado para dar tratamiento fiscal especial. Tipo: objeto.
*   **CM04**: puede marcar las operaciones bajo Convenio Multilateral (CM) o indicar que se trata de un vendedor inscripto en el CM. Tipo: objeto.
*   **CATEGORIA (Ajustado)**: variable `CATEGORIA` limpia (se redujo el número de valores posibles). Tipo: objeto.
*   **SUB-CATEGORIA**: variable `DESCRIPCION_CATEGORIA` limpia (se redujo el número de valores posibles). Tipo: objeto.
*   **NOMBRE**: desconocemos. Se trata de otra variable anonimizada mediante hasheo, pero es un hash diferente al de `ID_VENDEDOR`. Tipo: objeto.
*   **OMEGA**: marca para contribuyente de interés fiscal especial. Tipo: entero.
*   **MODELO**: identificación de vendedores modelo según la empresa proveedora de los datos. Tipo: entero.

In [ ]:
lista_columnas = list(ventas.columns)
print(len(lista_columnas), 'variables:')
lista_columnas

In [ ]:
ventas.dtypes

In [ ]:
display(ventas[:10])

### Estadística descriptiva

De las 19 variables, 9 son numéricas.
*   **INSCRIPCION**: al no conocer a qué hace referencia, no podemos interpretar la estadística descriptiva.
*   **DEPOSITO**: al no conocer a qué hace referencia, no podemos interpretar la estadística descriptiva.
*   **AÑO**: desde 2019 hasta 2022.
*   **MES**: los 12 meses del año.
*   **PORCENTAJE_COMISION_EMPRESA**: desde el 0 hasta el 18%.
*   **OMEGA**: todos tienen valor igual a 1, por lo que esta variable no aporta nada de información.
*   **MODELO**: se tienen valores 0 y 1 para identificar vendedores no modelo de vendedores modelo, respectivamente.
*   **TOTAL_VENTAS**: la dispersión de datos es enorme: la desviación estándar es un orden mayor que la media. Inlcuso hay valores negativos: hay que determinar si están mal imputados o si el hecho de ser negativos tiene un significado. Analizando los z-Scores vemos que el mínimo está a 5 mientras que el máximo está a 101 desviaciones estándares respecto a la media.
*   **COMISION_EMPRESA**: observaciones análogas al punto anterior. Analizando los z-Scores vemos que el mínimo está a 3 mientras que el máximo está a 131 desviaciones estándares respecto a la media.

In [ ]:
ventas.describe().T

In [ ]:
# TOTAL_VENTAS
zScore_min = (ventas['TOTAL_VENTAS'].min() - ventas['TOTAL_VENTAS'].mean()) / ventas['TOTAL_VENTAS'].std()
zScore_max = (ventas['TOTAL_VENTAS'].max() - ventas['TOTAL_VENTAS'].mean()) / ventas['TOTAL_VENTAS'].std()

print('z-Score de los extremos de la variable "TOTAL_VENTAS":')
print(f'\t Mínimo: {zScore_min:.0f}')
print(f'\t Mínimo: {zScore_max:.0f}')

# COMISION_EMPRESA
zScore_min = (ventas['COMISION_EMPRESA'].min() - ventas['COMISION_EMPRESA'].mean()) / ventas['COMISION_EMPRESA'].std()
zScore_max = (ventas['COMISION_EMPRESA'].max() - ventas['COMISION_EMPRESA'].mean()) / ventas['COMISION_EMPRESA'].std()

print('z-Score de los extremos de la variable "COMISION_EMPRESA":')
print(f'\t Mínimo: {zScore_min:.0f}')
print(f'\t Mínimo: {zScore_max:.0f}')

### Datos faltantes

Vemos que tenemos un total de 431.506 registros. Entre las variables disponibles tenemos 5 con valores faltantes:
* **CM04:** sólo hay 42 registros (menos del 0.01% del total de registros).
* **TRATAMIENTO_DIFERNCIAL:** tiene 83.058 reigstros (cerca del 20% del total de registros).
* **DESC_TRATAMIENTO_FISCAL:** tiene 117.841 registros (cerca del 30% del total de registros).
* **TRATAMIENTO_FISCAL:** tiene 403.538 registros (más del 90% del total de registros).
* **NOMBRE:** tiene 430.857 registros (casi el total de registros).

In [ ]:
msno.bar(ventas,figsize=(12, 3), sort="ascending",fontsize=12, color='gray')
plt.show()

### Cardinalidad: valores únicos

`ID_VENDEDOR` tiene 3209 valores (vendedores) diferentes. El vendedor mayoritario contribuye en un 0.39% de los registros y los 10 primeros en conjunto contribuyen al 2.85%.

In [ ]:
m = 0
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`INSCRIPCION` tiene 3345 valores diferentes. El valor mayoritario contribuye en un 0.39% de los registros y los 10 primeros en conjunto contribuyen al 2.72%. Hay 136 valores más que en `ID_VENDEDOR`, pero porcentajes similares: puede existir una relación entre ambas, la cual no necesariamente es biyectiva.

In [ ]:
m = 1
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`AÑO` tiene 4 valores diferentes: los 4 años que ya dijimos. El 2021 contribuye en un 29.69% de los registros, mientras que el 2022 con 15.02%.

In [ ]:
m = 2
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`MES` tiene 12 valores diferentes: los 12 meses del año. Junio es el que más contribuye con el 9.56% de los registros. Los 10 meses con mayores contribuciones suman un total del 85.78% de registros.

In [ ]:
m = 3
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`CATEGORIA` tiene 5 valores diferentes, pero parece que en realidad son todas la misma categoría, con diferentes formatos de escritura.

In [ ]:
m = 4
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`DEPOSITO` tiene 241 valores diferentes. El valor mayoritario contribuye en un 5.21% de los registros y los 10 primeros en conjunto contribuyen al 26.64%.

In [ ]:
m = 5
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`DESCRIPCION_CATEGORIA` tiene 245 valores diferentes. El valor mayoritario contribuye en un 5.21% de los registros y los 10 primeros en conjunto contribuyen al 26.64%. La cantidad de valores es similar al de la variable `DEPOSITO` y, además, las contribuciones porcentuales son simialres. Puede existir una relación entre ambas variables.

In [ ]:
m = 6
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`TOTAL_VENTAS` tiene 248974 valores diferentes. El valor mayoritario contribuye en un 41.96% de los registros, indicando ventas por un valor de $0. Esto es sospechoso: ¿es un indicio para vendedores que hacen fuga de ventas?. Por otro lado, los 10 primeros en conjunto contribuyen al 42.06%.

In [ ]:
m = 7
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`PORCENTAJE_COMISION_EMPRESA` tiene 89 valores diferentes. El valor mayoritario (4.75%) contribuye en un 19.41% de los registros y los 10 primeros en conjunto contribuyen al 73.27%.

In [ ]:
m = 8
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`COMISION_EMPRESA` tiene 244284 valores diferentes. El valor mayoritario contribuye en un 42.52% de los registros y los 10 primeros en conjunto contribuyen al 42.56%. Nuevamente el valor mayoritario es de $0 como en `TOTAL_VENTAS`. Tiene sentido que estas variables estén relacionadas.

In [ ]:
m = 9
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`TRATAMIENTO_FISCAL` tiene 17 valores diferentes: algunos numéricos y otros categóricos. El valor mayoritario contribuye en un 45.50% de los registros y los 10 primeros en conjunto contribuyen al 99.16%. Hay que unificar cosas como 0 y 0.0. Además ¿existe alguna relación entre ser 1 y ser especial 1? (o similares)

In [ ]:
m = 10
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`DESC_TRATAMIENTO_FISCAL` tiene 4 valores diferentes. El valor mayoritario contribuye en un 94.36% de los registros. ¿Qué relación hay entre esta variable y `TRATAMIENTO_FISCAL`?

In [ ]:
m = 11
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`TRATAMIENTO_DIFERNCIAL` tiene 11 valores diferentes. El valor mayoritario (Art. 21) contribuye en un 34.45% de los registros y los 10 primeros en conjunto contribuyen al 99.52%. ¿Cómo tratar a los casos "Art. 19", "Art. 20" y "Art. 19 y 20"?

In [ ]:
m = 12
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`CM04` tiene un único valor: "Si". Esta variable tenía casi el 100% de registros vacíos: ¿estar vacío implica un "No"?

In [ ]:
m = 13
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`CATEGORIA (Ajustado)` tiene un único valor: en efecto la variable `CATEGORIA` presentaba un único valor, escrito de formas diferentes.

In [ ]:
m = 14
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`SUB-CATEGORIA` tiene 21 valores diferentes: las 245 opciones de `DESCRIPCION_CATEGORIA` fueron reducidas a esta cantidad. El valor mayoritario contribuye en un 32.51% de los registros y los 10 primeros en conjunto contribuyen al 87.78%. Si existía una relación con `DEPOSITO` ¿se sigue apreciando en esta nueva variable?

In [ ]:
m = 15
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`NOMBRE` tiene 3199 valores diferentes. El valor mayoritario contribuye en un 0.39% de los registros y los 10 primeros en conjunto contribuyen al 2.85%. Si bien los porcentajes son similares a aquellos encontrados para `ID_VENDEDOR`, los hashs son diferentes y, además, hay 10 nombres menos.

In [ ]:
m = 16
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`OMEGA` tiene un único valor como habíamos dicho, por lo que no aporta nada de información relevante.

In [ ]:
m = 17
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`MODELO` tiene 2 valores diferentes como habíamos dicho. Sólo el 0.32% de los registros es considerado **vendedor modelo** por parte del cliente.

In [ ]:
m = 18
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

### Variables que no contribuyen

En base a lo visto hasta aquí, se podría prescindir de las variables `CATEGORIA` y `DESCRIPCION_CATEGORIA`, las cuales fueron superadas por `CATEGORIA (Ajustado)` y `SUB-CATEGORIA`, respectivamente. Sin embargo, como `CATEGORIA (Ajustado)` presenta un único valor posible, también es prescindible. Lo mismo ocurre con `OMEGA`.

In [ ]:
ventas_clean = ventas.drop(['CATEGORIA', 'DESCRIPCION_CATEGORIA', 'OMEGA', 'CATEGORIA (Ajustado)'], axis=1).copy()
display(ventas_clean[:3])

### Construcción de una nueva variable: Fecha

Consideramos que para el análisis temporal habrá momentos en que necesitamos tener año y mes por separado, pero en otras ocasiones necesitamos toda la información al mismo tiempo. Por lo tanto, creamos la variable fecha:
    $$FECHA = AÑO + MES$$

In [ ]:
ventas_clean['FECHA'] = pd.to_datetime(ventas_clean['MES'].astype(str) + '-' + ventas_clean['AÑO'].astype(str), format='%m-%Y')
display(ventas_clean[:3])

In [ ]:
ventas_clean['FECHA'].dtypes

### Simplificación de valores únicos

#### Unificacón de categorías de `TRATAMIENTO_FISCAL`

Parece que la variable asume valores enteros, floats y strings. Sin embargo, se corrobora que los que parecen enteros, en realidad son strings. Vamos a forzar los floats hacia strings.

In [ ]:
ventas_clean['TRATAMIENTO_FISCAL'].value_counts()

In [ ]:
ventas_clean['TRATAMIENTO_FISCAL'].unique()

In [ ]:
ventas_clean['TRATAMIENTO_FISCAL'] = ventas_clean['TRATAMIENTO_FISCAL'].replace({0.0: '0', 3.0: '3', 2.0: '2', 1.0: '1'})
ventas_clean['TRATAMIENTO_FISCAL'].value_counts()

#### Simplificación de categorías de `DESC_TRATAMIENTO_FISCAL`

Simplificamos los nombres de las categorías para que sea vea mejor a la hora de graficar.

In [ ]:
ventas_clean['DESC_TRATAMIENTO_FISCAL'].value_counts()

In [ ]:
ventas_clean['DESC_TRATAMIENTO_FISCAL'] = ventas_clean['DESC_TRATAMIENTO_FISCAL'].replace({'Normal': 'Norm', 'Otro Tratamiento Fiscal': 'Otro', 'Exento/Desgravado': 'Ex/Des', 'Minorista': 'Min'})
ventas_clean['DESC_TRATAMIENTO_FISCAL'].value_counts()

#### Simplificación de categorías de `TRATAMIENTO_DIFERENCIAL`

Simplificamos el nombre de los arítculos. Falta ver qué hacer con los casos que responden tanto al artículo 19 como al 20.

In [ ]:
ventas_clean['TRATAMIENTO_DIFERNCIAL'].value_counts()

In [ ]:
ventas_clean['TRATAMIENTO_DIFERNCIAL'] = ventas_clean['TRATAMIENTO_DIFERNCIAL'].replace({'Artículo 21': 'Art.21',
                                                                                         'Artículo 20': 'Art.20',
                                                                                         'Artículo 19 y 20': 'Art.19+20',
                                                                                         'Artículo 22': 'Art.22',
                                                                                         'Artículo 16': 'Art.16',
                                                                                         'Artículo 18': 'Art.18',
                                                                                         'Artículo 34': 'Art.34',
                                                                                         'Artículo 19': 'Art.19',
                                                                                         'Artículo 31': 'Art.31',
                                                                                         'Artículo 17': 'Art.17',
                                                                                         'Artículo 28': 'Art.28'})
ventas_clean['TRATAMIENTO_DIFERNCIAL'].value_counts()

### Algunas pruebas

#### Significado de los registros

Para saber qué representa cada registro (ventas mensuales, vendedores por mes, etcétera). Vemos que hay más de una observación para cada fecha por vendedor. Probablemente cada registro represente operaciones de ventas individuales.

In [ ]:
prueba1 = ventas_clean[['ID_VENDEDOR', 'FECHA']].copy()

In [ ]:
for k in range(len(prueba1['ID_VENDEDOR'])):
    p = prueba1[prueba1['ID_VENDEDOR']==prueba1['ID_VENDEDOR'][k]]
    p = np.sum(p.duplicated(subset='FECHA'))
    if p != 0:
        print('Existen vendedores con más de un registro en la misma "FECHA".')
        break

In [ ]:
# Vamos a eliminar duplicados por ID_VENDEDOR y FECHA
prueba1 = ventas_clean.drop_duplicates(subset=['ID_VENDEDOR', 'FECHA']).copy()
print(len(prueba1))
print(len(ventas_clean))

#### Relación entre `ID_VENDEDOR` y `NOMBRE`

Queremos saber si existe una corrspondencia entre estas 2 variables. Ya vimos que había 10 nombres menos. Ahora vemos que hay más de un registro que asocia un dado `ID_VENDEDOR` con cierto `NOMBRE`. Falta saber qué representa `NOMBRE`para saber si se puede descartar o no.

In [ ]:
prueba2 = ventas_clean[['ID_VENDEDOR', 'NOMBRE']].copy()

In [ ]:
for k in range(len(prueba2['ID_VENDEDOR'])):
    p = prueba2[prueba2['ID_VENDEDOR']==prueba2['ID_VENDEDOR'][k]]
    p = np.sum(p.duplicated(subset='NOMBRE'))
    if p != 0:
        print('Existen vendedores con más de un registro con el mismo "NOMBRE".')
        break

#### Relación entre `TRATAMIENTO_FISCAL` y `DESC_TRATAMIENTO_FISCAL`

Queremos chequear para qué valores de `TRATAMIENTO_FISCAL` aparecen valores de `DESC_TRATAMIENTO_FISCAL`. Al analizar la tabla de contingencia, vemos que los valores de `DESC_TRATAMIENTO_FISCAL` sólo aparecen cuando `TRATAMIENTO_FISCAL` asume valores 0, 1, 2 ó 3. A su vez, hay correspondencia entre 0 y Normal, 1 y Exento/Desgravado, 2 y Minorista, 3 y Otro Tratamiento Fiscal. ¿Se puede unificar todo como `TRATAMIENTO_FISCAL`, eliminando entonces `DESC_TRATAMIENTO_FISCAL`? ¿Tiene sentido que falte la descripción asociada al tratamiento?

In [ ]:
# Este paso lo agrego porque sino el crosstab siguiente no me muestra los NaN
prueba3 = ventas_clean.copy()
prueba3['TRATAMIENTO_FISCAL'] = prueba3['TRATAMIENTO_FISCAL'].fillna('')
prueba3['DESC_TRATAMIENTO_FISCAL'] = prueba3['DESC_TRATAMIENTO_FISCAL'].fillna('')

In [ ]:
pd.crosstab(prueba3['TRATAMIENTO_FISCAL'], prueba3['DESC_TRATAMIENTO_FISCAL'], dropna=False)

#### Relación entre `TRATAMIENTO_FISCAL` y `TRATAMIENTO_DIFERENCIAL`

Ahora veamos para qué valores de `TRATAMIENTO_FISCAL` aparecen valores de `TRATAMIENTO_DIFERNCIAL`. Al analizar la tabla de contingencia vemos que no hay un patrón claro. ¿Podríamos tomar a un dato faltante como "sin tratamiento" o algo por el estilo?

In [ ]:
# Este paso lo agrego porque sino el crosstab siguiente no me muestra los NaN
prueba3['TRATAMIENTO_DIFERNCIAL'] = prueba3['TRATAMIENTO_DIFERNCIAL'].fillna('')

In [ ]:
pd.crosstab(prueba3['TRATAMIENTO_FISCAL'], prueba3['TRATAMIENTO_DIFERNCIAL'], dropna=False)

#### Relación descriptiva entre `TOTAL_VENTAS`, `PORCENTAJE_COMISION_EMPRESA` y `COMISION_EMPRESA`

Como ya vimos, existen valores negativos en `TOTAL_VENTAS` y `COMISION_EMPRESA`. ¿Tiene algún significado el signo negativo o es un error? También hay muchos registros nulos: ¿tiene algún significado especial o es un indicio de fuga? 

Destacamos nuevamente que hay una dispersión muy grande, presentándose la mayor asimetría hacia la derecha.

Vemos que poco más del 42% de los registros poseen un valor negativo o nulo en alguna de estas 2 variables.

Analizando los porcentajes de comisión, ¿tiene sentido que haya porcentajes nulos?

Hay que decidir:
* Qué hacemos con valores negativos.
* Qué hacemos con valores nulos.
* Si truncamos valores extremos y, en caso afirmativo, la manera/los límites.

In [ ]:
ventas_clean[['TOTAL_VENTAS','PORCENTAJE_COMISION_EMPRESA','COMISION_EMPRESA']].describe()

In [ ]:
# TOTAL_VENTAS
ans = len(ventas_clean)
neg = len(ventas_clean[ventas_clean["TOTAL_VENTAS"] < 0])
nul = len(ventas_clean[ventas_clean["TOTAL_VENTAS"] == 0])
print(f'Valores negativos:')
print(f'\t{neg} de {ans}.')
print(f'\t{100*neg/ans:.2f}%')

print(f'Valores nulos:')
print(f'\t{nul} de {ans}.')
print(f'\t{100*nul/ans:.2f}%')

print(f'Valores negativos más nulos:')
print(f'\t{nul+neg} de {ans}.')
print(f'\t{100*(nul+neg)/ans:.2f}%')

In [ ]:
# COMISION_EMPRESA
ans = len(ventas_clean)
neg = len(ventas_clean[ventas_clean["COMISION_EMPRESA"] < 0])
nul = len(ventas_clean[ventas_clean["COMISION_EMPRESA"] == 0])
print(f'Valores negativos:')
print(f'\t{neg} de {ans}.')
print(f'\t{100*neg/ans:.2f}%')

print(f'Valores nulos:')
print(f'\t{nul} de {ans}.')
print(f'\t{100*nul/ans:.2f}%')

print(f'Valores negativos más nulos:')
print(f'\t{nul+neg} de {ans}.')
print(f'\t{100*(nul+neg)/ans:.2f}%')

In [ ]:
# Crear la figura y los subplots
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

# Boxplot para 'TOTAL_VENTAS'
axs[0].boxplot(ventas_clean['TOTAL_VENTAS'].dropna())
axs[0].set_title('Boxplot de TOTAL_VENTAS')

# Boxplot para 'COMISION_EMPRESA'
axs[1].boxplot(ventas_clean['COMISION_EMPRESA'].dropna())
axs[1].set_title('Boxplot de COMISION_EMPRESA')

# Boxplot para 'PORCENTAJE_COMISION_EMPRESA'
axs[2].boxplot(ventas_clean['PORCENTAJE_COMISION_EMPRESA'].dropna())
axs[2].set_title('Boxplot de PORCENTAJE_COMISION_EMPRESA')

# Ajustar los espacios entre subplots
plt.tight_layout()

# Mostrar los gráficos
plt.show()

In [ ]:
prueba4 = ventas_clean[['TOTAL_VENTAS','PORCENTAJE_COMISION_EMPRESA','COMISION_EMPRESA']]

En los histogramas de la diagonal principal podemos apreciar el enorme peso de los valores nulos sobre el total de registros. Luego:
* La mayoría de `TOTAL_VENTAS` están asociados a comisiones menores al 5%.
* Se proyectan diferentes rectas entre `TOTAL_VENTAS` y `COMISION_EMPRESA`: ¿existe una relación de proporcionaldiad con el porcentaje?
* Notando la anterior relación, es inmediato observar que la mayoría de `COMISION_EMPRESA` están asociados también a comisiones menores al 5%.

In [ ]:
sns.pairplot(data=prueba4, diag_kind='kde', plot_kws = {'alpha': 0.3, 's': 10})
plt.show()

#### Relación numérica entre `TOTAL_VENTAS`, `PORCENTAJE_COMISION_EMPRESA` y `COMISION_EMPRESA`

Queremos estudiar un poco más la relación lineal antes vista entre estas 3 variables. Proponemos que la función es
    $$\text{COMISION\_EMPRESA} = \text{PORCENTAJE\_COMISION\_EMPRESA} \times \text{TOTAL\_VENTAS}$$

Vemos que la fórmula es exacta sólo para 190.999 registros. Si probamos con algún margen de error, la fórmula verifica con precisión menor a 1 entero para 261.272 registros (menos del 61% de los registros totales).

In [ ]:
# Probemos con verificar la siguiente fórmula:
ventas_clean['ComCalc'] = ventas_clean['TOTAL_VENTAS'] * ventas_clean['PORCENTAJE_COMISION_EMPRESA']

In [ ]:
for x in [0, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 5, 10, 20, 50, 100, 1000]:
  print('Para', x, 'de diferencia, tenemos', sum(abs(ventas_clean['COMISION_EMPRESA'] - ventas_clean['ComCalc']) <= x), 'igualdades')

Tenemos que en el 44.26% de los casos la relación lineal que proponemos es exacta. Respecto a la fracción restante, en el 8.19% de los casos la subestima, mientras que en el 47.55% la sobreestima.

In [ ]:
diferencia = ventas_clean['COMISION_EMPRESA'] - ventas_clean['ComCalc']
print(f'Exactos: {100*len(diferencia[diferencia==0])/len(diferencia):.2f}')
print(f'Positivos: {100*len(diferencia[diferencia>0])/len(diferencia):.2f}')
print(f'Negativos: {100*len(diferencia[diferencia<0])/len(diferencia):.2f}')

In [ ]:
# Grafiquemos la distribución de la diferencia
plt.boxplot(diferencia.dropna())
plt.xlabel('Diferencia')
plt.ylabel('Valor')
plt.title('Diferencia entre comisión empresa real y calculada')
plt.show()

Vemos que, a pesar de las diferencias entre el valor real y el predicho, se pueden apreciar claramente 2 pendientes: una muy grande y otra bastante más pequeña. Esas dos rectas serían las que describen mejor la tendencia de los puntos y, por lo tanto, explicarían la relación entre las 3 variables de interés.

In [ ]:
com_real_sobre = ventas_clean['COMISION_EMPRESA'][diferencia<0]
com_real_exac = ventas_clean['COMISION_EMPRESA'][diferencia==0]
com_real_sub = ventas_clean['COMISION_EMPRESA'][diferencia>0]

com_calc_sobre = ventas_clean['ComCalc'][diferencia<0]
com_calc_exac = ventas_clean['ComCalc'][diferencia==0]
com_calc_sub = ventas_clean['ComCalc'][diferencia>0]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].scatter(com_real_sobre, com_calc_sobre, label='Sobreestimado', s=30)
axs[0].scatter(com_real_exac, com_calc_exac, label='Exacto', s=20)
axs[0].scatter(com_real_sub, com_calc_sub, label='Subestimado', s=10)

axs[0].legend()

axs[0].set_xlabel('Comisión real')
axs[0].set_ylabel('Comisión Calculada')

axs[1].scatter(com_real_sobre, com_calc_sobre, label='Sobreestimado', s=30)
axs[1].scatter(com_real_exac, com_calc_exac, label='Exacto', s=20)
axs[1].scatter(com_real_sub, com_calc_sub, label='Subestimado', s=10)

axs[1].legend()
axs[1].set_ylim(-5E8, 1E8)
axs[1].set_xlim(-0.5E7, 3.5E7)

axs[1].set_xlabel('Comisión real')
axs[1].set_ylabel('Comisión calculada')

fig.suptitle('Comisión empresa: real vs calculada')
plt.show()

Vemos que entonces tenemos 2 rectas que definen la relación entre las variables de interés. Por un lado, el 61% de los datos responde a la ecuación antes planteada, sumando un pequeño offset:
    $$\text{Com. Calc} = \text{Com. Real} + \$0.48 = \text{Porcent. Com.} \times \text{Ventas}$$
    $$\Rightarrow \text{Com. Real} = \text{Porcent. Com.} \times \text{Ventas} - \$0.48$$

El resto de los datos responde a una ecuación completamente distinta:
    $$\text{Com. Calc} = 100 * \text{Com. Real} + \$1365 = \text{Porcent. Com.} \times \text{Ventas}$$
    $$\Rightarrow \text{Com. Real} = \frac{\text{Porcent. Com.} \times \text{Ventas} - \$1365}{100} = 0.01 \times \text{Porcent. Com.} \times \text{Ventas} - \$13.65 $$

No olvidar que esto sale considerando absolutamente todos los valores (no se descaartó ni se truncó nada).

In [ ]:
rg_real = pd.concat([com_real_sub[com_calc_sub<0], com_real_sobre[com_calc_sobre>10*com_real_sobre]])
rg_calc = pd.concat([com_calc_sub[com_calc_sub<0], com_calc_sobre[com_calc_sobre>10*com_real_sobre]])

rp_real = pd.concat([com_real_sub[com_calc_sub>0], com_real_sobre[com_calc_sobre<10*com_real_sobre], com_real_exac])
rp_calc = pd.concat([com_calc_sub[com_calc_sub>0], com_calc_sobre[com_calc_sobre<10*com_real_sobre], com_calc_exac])

In [ ]:
print(len(rg_real))
print(len(rp_real))
print(len(rp_real+rg_real))

In [ ]:
LG = LR(rg_real, rg_calc)
LP = LR(rp_real, rp_calc)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs[0].scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs[0].plot(rg_real, rg_real*LG.slope + LG.intercept, label=f'{LG.slope:.0f}*Real + {LG.intercept:.0f}', color='red')
axs[0].plot(rp_real, rp_real*LP.slope + LP.intercept, label=f'{LP.slope:.0f}*Real + {LP.intercept:.2f}', color='tab:green')

axs[0].legend(loc='upper left')

axs[0].set_xlabel('Comisión real')
axs[0].set_ylabel('Comisión Calculada')

axs[1].scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs[1].scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs[1].plot(rg_real, rg_real*LG.slope + LG.intercept, label=f'{LG.slope:.0f}*Real + {LG.intercept:.0f}', color='red')
axs[1].plot(rp_real, rp_real*LP.slope + LP.intercept, label=f'{LP.slope:.0f}*Real + {LP.intercept:.2f}', color='tab:green')

axs[1].legend(loc='lower right')
axs[1].set_ylim(-1E8, 1E8)
axs[1].set_xlim(-0.5E7, 4E7)

axs[1].set_xlabel('Comisión real')
axs[1].set_ylabel('Comisión calculada')

fig.suptitle('Comisión empresa: real vs calculada')
plt.show()

#### Rectas obtenida vs vendedor modelo

Ahora comparamos las dos rectas obtenidas en función de la clasificación de modelo dada por el cliente. Vemos que todos los vendedores modelo caen dentro de la recta de menor pendiente, la cual establece prácticamente una relación de proporcionalidad directa entre las ventas y la comisión de la empresa. Por su parte, los que no están clasificados como modelo caen sobre ambas rectas. ¿Puede ser este un camino para detectar la fuga?

In [ ]:
mask0 = ventas_clean['MODELO']==0
mask1 = ventas_clean['MODELO']==1

In [ ]:
com_real_Mod = ventas_clean['COMISION_EMPRESA'][mask1]
com_real_noMod = ventas_clean['COMISION_EMPRESA'][mask0]

com_calc_Mod = ventas_clean['ComCalc'][mask1]
com_calc_noMod = ventas_clean['ComCalc'][mask0]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs[0].scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs[0].plot(com_real_noMod, com_calc_noMod, label='No Modelo', color='tab:green')
axs[0].plot(com_real_Mod, com_calc_Mod, label='Modelo', color='red')

axs[0].legend(loc='upper left')

axs[0].set_xlabel('Comisión real')
axs[0].set_ylabel('Comisión Calculada')

axs[1].scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs[1].scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs[1].plot(com_real_noMod, com_calc_noMod, label='No Modelo', color='tab:green')
axs[1].plot(com_real_Mod, com_calc_Mod, label='Modelo', color='red')

axs[1].legend(loc='lower right')
axs[1].set_ylim(-1E8, 1E8)
axs[1].set_xlim(-0.5E7, 4E7)

axs[1].set_xlabel('Comisión real')
axs[1].set_ylabel('Comisión calculada')

fig.suptitle('Comisión empresa: real vs calculada')
plt.show()

### Acomodado del DataFrame resultante

Quitamos la columan que creamos recién con los cálculos y reordenamos las columnas, poniendo las más relevantes al comienzo.

In [ ]:
# Eliminamos la columna creada
ventas_clean = ventas_clean.drop(['ComCalc'], axis=1).copy()
ventas_clean.columns

In [ ]:
ventas_clean = ventas_clean[['FECHA', 'MODELO', 'TOTAL_VENTAS', 
                             'PORCENTAJE_COMISION_EMPRESA', 'COMISION_EMPRESA',
                             'SUB-CATEGORIA', 'ID_VENDEDOR', 'NOMBRE',
                             'INSCRIPCION', 'DEPOSITO', 'AÑO', 'MES',
                             'TRATAMIENTO_FISCAL', 'DESC_TRATAMIENTO_FISCAL',
                             'TRATAMIENTO_DIFERNCIAL', 'CM04']]
ventas_clean = ventas_clean.sort_values(['FECHA', 'TOTAL_VENTAS'])

In [ ]:
ventas_clean

---
# Paso 2: Análisis estadístico descriptivo

### Descripción general, sin considerar serie temporal

Vamos a considerar que la serie temporal **principal** es la dictada por `TOTAL_VENTAS`. Recordamos primero la estadística descriptiva general. Como dijimos antes, hay una gran dispersión en los datos de `TOTAL_VENTAS` y `COMISION_EMPRESA`, las cuales ya sabemos que están correlacionadas.

In [ ]:
ventas_clean.describe().T

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

sns.histplot(ventas_clean['AÑO'], ax=axs[0])
sns.histplot(ventas_clean['MES'], ax=axs[1])
sns.histplot(ventas_clean['FECHA'], ax=axs[2])
axs[2].tick_params(axis='x', rotation=90)

plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

sns.boxplot(x=ventas_clean['TOTAL_VENTAS'], ax=axs[0, 0], notch=True, 
            flierprops={"marker": "x"}, medianprops={"color": "coral"})

sns.boxplot(x=ventas_clean['TOTAL_VENTAS'], ax=axs[0, 1], notch=True, 
            flierprops={"marker": "x"}, medianprops={"color": "coral"})

axs[0, 1].set_xlim(-0.6E7, 1E7)

sns.boxplot(x=ventas_clean['COMISION_EMPRESA'], ax=axs[1, 0], notch=True, 
            flierprops={"marker": "x"}, medianprops={"color": "coral"})

sns.boxplot(x=ventas_clean['COMISION_EMPRESA'], ax=axs[1, 1], notch=True, 
            flierprops={"marker": "x"}, medianprops={"color": "coral"})

axs[1, 1].set_xlim(-3E5, 4E5)

plt.show()

Ya habíamos dicho que teníamos valores extremos tanto por izquierda como por derecha, siendo mucho más pronunciada la asimetría hacia la derecha. También observamos los valores negativos que aún no sabemos si son erróneos o si el signo tiene algún significado.

Para atacar el problema de los valores extremos podemos proponer quedarnos con cierta porción de la distribución usando percentiles. Como tiene mucha cola hacia la derecha, nos quedamos primero con la porción izquierda de la población.

In [ ]:
print(f'Teníamos {len(ventas_clean)} registros.')
dftest = ventas_clean.copy()

fig, axs = plt.subplots(1, 4, figsize=(24, 5))

sns.boxplot(x=ventas_clean['TOTAL_VENTAS'], ax=axs[0], notch=True, 
            flierprops={"marker": "x"}, medianprops={"color": "coral"})

percentile_sup = dftest['TOTAL_VENTAS'].quantile(0.995)
dfk = dftest[dftest['TOTAL_VENTAS'] < percentile_sup]
print(f'Tomando el 99.5% central quedan {len(dfk)} registros.')
sns.boxplot(x=dfk['TOTAL_VENTAS'], ax=axs[1], notch=True, 
            flierprops={"marker": "x"}, medianprops={"color": "coral"})

percentile_sup = dftest['TOTAL_VENTAS'].quantile(0.990)
dfk = dftest[dftest['TOTAL_VENTAS'] < percentile_sup]
print(f'Tomando el 99.0% central quedan {len(dfk)} registros.')
sns.boxplot(x=dfk['TOTAL_VENTAS'], ax=axs[2], notch=True, 
            flierprops={"marker": "x"}, medianprops={"color": "coral"})

percentile_sup = dftest['TOTAL_VENTAS'].quantile(0.950)
dfk = dftest[dftest['TOTAL_VENTAS'] < percentile_sup]
print(f'Tomando el 95.0% central quedan {len(dfk)} registros.')
sns.boxplot(x=dfk['TOTAL_VENTAS'], ax=axs[3], notch=True, 
            flierprops={"marker": "x"}, medianprops={"color": "coral"})

axs[0].set_title(f'100% de la distribución')
axs[1].set_title(f'99.5% izquierdo de la distribución')
axs[2].set_title(f'99.0% izquierdo de la distribución')
axs[3].set_title(f'95.0% izquierdo de la distribución')

plt.show()

### Considerando la serie temporal

#### Agrupando por meses

Se entiende más la progresión al agrupar todos los datos correspondientes a un mismo mes. Vemos que no hace otra cosa más que crecer. Se observa el efecto de la inflación y pareciera haber picos hacia fin de año.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].plot(ventas_clean['FECHA'],ventas_clean['TOTAL_VENTAS'])

axs[0].set_xlabel('Fecha')
axs[0].set_ylabel('Ventas')
axs[0].set_title('Ventas sin agregar')

prueba6 = ventas_clean[['FECHA','TOTAL_VENTAS', 'AÑO']].copy()
prueba6_agregado = prueba6.groupby('FECHA')['TOTAL_VENTAS'].sum().reset_index()

axs[1].plot(prueba6_agregado['FECHA'], prueba6_agregado['TOTAL_VENTAS'])

axs[1].set_xlabel('Fecha')
axs[1].set_ylabel('Ventas')
axs[1].set_title('Ventas agregadas por mes')

plt.show()

Son más similares las tendencias desde el 2019 al 2021, mientras que 2022 se destaca por ser bien diferente.

In [ ]:
mes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

v19 = prueba6_agregado['TOTAL_VENTAS'][:12]
v20 = prueba6_agregado['TOTAL_VENTAS'][12:24]
v21 = prueba6_agregado['TOTAL_VENTAS'][24:36]
v22 = prueba6_agregado['TOTAL_VENTAS'][36:]

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].plot(mes, v19, label='2019', marker='o')
axs[0].plot(mes, v20, label='2020', marker='o')
axs[0].plot(mes, v21, label='2021', marker='o')
axs[0].plot(mes[:6], v22, label='2022', marker='o')

axs[0].set_xlabel('Mes')
axs[0].set_ylabel('Ventas')
axs[0].set_title('Original')
axs[0].legend()

axs[1].plot(mes, v19/np.max(v19), label='2019', marker='o')
axs[1].plot(mes, v20/np.max(v20), label='2020', marker='o')
axs[1].plot(mes, v21/np.max(v21), label='2021', marker='o')
axs[1].plot(mes[:6], v22/np.max(v22), label='2022', marker='o')

axs[1].set_xlabel('Mes')
axs[1].set_ylabel('Ventas')
axs[1].set_title('Normalizadas')
axs[1].legend()

fig.suptitle('Ventas agregadas por mes y separadas por año')

plt.show()

#### Estacionalidad un poco más definida - mensual

Se dice que una serie de tiempo es estacionaria si no aumenta o disminuye con el tiempo de forma lineal o exponencial (sin tendencias), y si no muestra ningún tipo de patrón repetitivo (sin estacionalidad). Matemáticamente, esto se describe como una media constante y una varianza constante a lo largo del tiempo. Junto con la varianza, la autocovarianza tampoco debería ser una función del tiempo.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 10))

sns.boxplot(data=ventas_clean, x='MES', y='TOTAL_VENTAS', ax=axs[0])

sns.boxplot(data=ventas_clean, x='MES', y='TOTAL_VENTAS', ax=axs[1])
axs[1].set_ylim(-0.6E7, 1.2E7)

fig.suptitle('Estacionalidad mensual considerando los 4 años juntos')
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(24, 20))

sns.boxplot(data=ventas_clean[ventas_clean['AÑO']==2019], x='MES', y='TOTAL_VENTAS', ax=axs[0, 0])
sns.boxplot(data=ventas_clean[ventas_clean['AÑO']==2019], x='MES', y='TOTAL_VENTAS', ax=axs[1, 0])

sns.boxplot(data=ventas_clean[ventas_clean['AÑO']==2020], x='MES', y='TOTAL_VENTAS', ax=axs[0, 1])
sns.boxplot(data=ventas_clean[ventas_clean['AÑO']==2020], x='MES', y='TOTAL_VENTAS', ax=axs[1, 1])

sns.boxplot(data=ventas_clean[ventas_clean['AÑO']==2021], x='MES', y='TOTAL_VENTAS', ax=axs[0, 2])
sns.boxplot(data=ventas_clean[ventas_clean['AÑO']==2021], x='MES', y='TOTAL_VENTAS', ax=axs[1, 2])

sns.boxplot(data=ventas_clean[ventas_clean['AÑO']==2022], x='MES', y='TOTAL_VENTAS', ax=axs[0, 3])
sns.boxplot(data=ventas_clean[ventas_clean['AÑO']==2022], x='MES', y='TOTAL_VENTAS', ax=axs[1, 3])

axs[0, 0].set_title('2019')
axs[1, 0].set_ylim(-0.6E7, 1.2E7)

axs[0, 1].set_title('2020')
axs[1, 1].set_ylim(-0.6E7, 1.2E7)

axs[0, 2].set_title('2021')
axs[1, 2].set_ylim(-0.6E7, 1.2E7)

axs[0, 3].set_title('2022')
axs[1, 3].set_ylim(-0.6E7, 1.2E7)

fig.suptitle('Estacionalidad mensual por año')
plt.show()

#### Estacionalidad un poco más definida - anual

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 10))

sns.boxplot(data=ventas_clean, x='AÑO', y='TOTAL_VENTAS', ax=axs[0])

sns.boxplot(data=ventas_clean, x='AÑO', y='TOTAL_VENTAS', ax=axs[1])
axs[1].set_ylim(-0.6E7, 1.2E7)

fig.suptitle('Estacionalidad anual considerando los 12 meses juntos')
plt.show()

#### Autocorrelación

Hay correlación estadísticamente significativa para los lags 1, 2 y 3, lo que implica que observaciones pasadas influyen en las observaciones actuales de la serie temporal de manera positiva.

In [ ]:
# Calcular la función de autocorrelación
acf = plot_acf(prueba6_agregado['TOTAL_VENTAS'], lags=12)

# Graficar la función de autocorrelación
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function')
plt.show()

---
# Paso 4: Distribución de frecuencias y probabilidades

1. Si los datos son continuos, crea un histograma para visualizar la distribución de frecuencias.
2. Calcula medidas como la asimetría y la curtosis para analizar la forma de la distribución.
3. Si los datos son discretos, crea una tabla de frecuencias y grafica un diagrama de barras para visualizar la distribución.